In [2]:
using Statistics
using DataStructures
using NNlib
using Graphs
using PyCall
nx = pyimport("networkx")
pyg = pyimport("torch_geometric")
include("src/weisfeiler_lehman.jl")
include("src/mpgnn.jl")
include("src/utils.jl")
colors = theme_palette(:default)

TO DO
- [ ] Experimento 1: Para los sacos, elegir los $p$ para que sea sparse. Elegir N y P para tener 200 - 300 grafos. 2 plots
- [ ] Experimento 2: Generar 1 grafo aleatorio N= 100: salto: 5000, con p = 0.01. Para cada grafo, correr gnn con bits variables desde 5-10 y subiendo. Para cada grafo, generar \gammas y calcular promedio de correctitud + var(). grafico bits/n. erdos renyi. 1 plot
- [] Experimento 3: Erdos N=5000. Hacer grafico correctitud / bits. 1plot
- [] Experimento 4: Repetir para Cora. 1plot

Enviar gráficos y parametros


## WL and MPGNN basic test

In [3]:
edge_index = [
     1 1 1 2 2 3 3 3 4 4 5 5;
     2 3 5 1 3 1 2 4 3 5 1 4
]
x₀ = [
     1, 1, 1, 1, 1
]
γ = 0.1
wl_iters, wl_coloring = WeisfeilerLehman(x₀, edge_index)
gnn_coloring = MPGNN(x₀, edge_index, γ, 3)

println(wl_coloring)
println(gnn_coloring)

BigFloat[1.0, 2.0, 1.0, 3.0, 3.0]
BigFloat[1.0, 2.0, 1.0, 3.0, 3.0]


![Alt Text](wl.gif)

## 1. Bag of Graphs Experiment
### 1.1 Bag of Erdős–Rényi

Here, we create an Erdős–Rényi random graph with **n** vertices. Edges are added between pairs of vertices with probability **p**.

In [4]:
setprecision(BigFloat, 51)
γs = sort([rand(BigFloat) for _ in 1:50])
lottery_γs = []
success_sum = []
P = [p for p in 0.05 : 0.01 : 0.1]
N = [n for n in 50 : 5 : 100]

graphs = [[erdos_renyi(N[j], P[i]) for j in 1:length(N)] for i ∈ 1:length(P)]
graphs = [[graphs[i][j][findall(>(0), degree(graphs[i][j]))] for j in 1:length(N)] for i ∈ 1:length(P)]

for γ ∈ γs
    success_matrix = zeros(length(P), length(N))
    for i ∈ 1:length(P)
        for j in 1:length(N)
            success_matrix[i, j] = WL_vs_GNN(graphs[i][j], γ)
        end
    end
    push!(success_sum, sum(success_matrix))
    if sum(success_matrix) == sum(ones(length(P), length(N)))
        # println("lottery ticket γ: $(γ)")
        push!(lottery_γs, γ) 
    end
end

ones_sum = sum(ones(length(P), length(N)))
selected_lottery_γs = vcat(lottery_γs[1], vcat(rand(lottery_γs, 3), lottery_γs[end]))
plot(γs, [ones_sum for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, success_sum, label="MPGNN")
scatter!(lottery_γs, [ones_sum for _ ∈ lottery_γs],
        mc=colors[3], ms=3, ma=0.5,
        label="Lottery γs")
scatter!(selected_lottery_γs, [ones_sum for _ ∈ selected_lottery_γs], 
        series_annotations = text.(selected_lottery_γs, :left, 8, rotation=270), 
        mc=colors[3], ms=3, ma=0.5,
        label="")

ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPGNN, Erdős Rényi")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\1.1.erdos_renyi.png")

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\Single-channel-GNN\\results\\1.1.erdos_renyi.png"

### 1.2 Bag of Barabási–Albert

Here, we create a Barabási–Albert model random graph with n vertices. It is grown by adding new vertices to an initial graph with k vertices. Each new vertex is attached with k edges to k different vertices already present in the system by preferential attachment. Initial graphs are undirected and consist of isolated vertices by default.

In [10]:
setprecision(BigFloat, 50)
γs = sort([rand(BigFloat) for _ in 1:50])
lottery_γs = []
success_sum = []
K = [k for k in 1 : 5 : 20]
N = [n for n in 20 : 10 : 100]

graphs = [[barabasi_albert(N[j], K[i]) for j in 1:length(N)] for i ∈ 1:length(K)]
graphs = [[graphs[i][j][findall(>(0), degree(graphs[i][j]))] for j in 1:length(N)] for i ∈ 1:length(K)]

for γ ∈ γs
    success_matrix = zeros(length(K), length(N))
    for i ∈ 1:length(K)
        for j in 1:length(N)
            success_matrix[i, j] = WL_vs_GNN(graphs[i][j], γ)
        end
    end
    push!(success_sum, sum(success_matrix))
    if sum(success_matrix) == sum(ones(length(K), length(N)))
        # println("lottery ticket γ: $(γ)")
        push!(lottery_γs, γ) 
    end
end

ones_sum = sum(ones(length(K), length(N)))
selected_lottery_γs = vcat(lottery_γs[1], vcat(rand(lottery_γs, 3), lottery_γs[end]))
plot(γs, [ones_sum for _ ∈ γs], label="WL", dpi=1000)
plot!(γs, success_sum, label="MPGNN")
scatter!(lottery_γs, [ones_sum for _ ∈ lottery_γs],
        mc=colors[3], ms=3, ma=0.5,
        label="Lottery γs")
scatter!(selected_lottery_γs, [ones_sum for _ ∈ selected_lottery_γs], 
        series_annotations = text.(selected_lottery_γs, :left, 8, rotation=270), 
        mc=colors[3], ms=3, ma=0.5,
        label="")

ylims!(0, 1.1*ones_sum)
title!("WL vs 1D-MPGNN, Barabási Albert")
xlabel!("γ")
ylabel!("successful simulations")
savefig("results\\1.2.barabasi_albert.png")

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\Single-channel-GNN\\results\\1.2.barabasi_albert.png"

## 2. Successful Bits Experiment 


Generar 1 grafo aleatorio N= 100: salto: 5000, con p = 0.01. Para cada grafo, correr gnn con bits variables desde 5-10 y subiendo. Para cada grafo, generar \gammas y calcular promedio de correctitud + var(). grafico bits/n. erdos renyi. 1 plot

Hacer gráfico que muestre la media y la desviación estandar considerando todos los gammas aleatorios


Hacer un plot de bits necesarios para alcanzar un optimo dado un grfo de tamaño n

In [11]:
# P = [p for p in 0.1 : 0.05 : 0.5]
P = [0.1]
N = [n for n in 50 : 50 : 400]

graphs = [[erdos_renyi(N[j], P[i]) for j in 1:length(N)] for i ∈ 1:length(P)]
graphs = [[graphs[i][j][findall(>(0), degree(graphs[i][j]))] for j in 1:length(N)] for i ∈ 1:length(P)]

bits = [i for i ∈ 10: 1 :60]
bits_success = [[] for _ in N]

for i ∈ 1:length(P)
    for j in 1:length(N)
        for sampling in 1:10
            for bit ∈ bits
                setprecision(BigFloat, bit)
                γ = rand(BigFloat)
                if WL_vs_GNN(graphs[i][j], γ)
                    push!(bits_success[j], bit)
                    break
                end
            end
        end
    end
end

regression = Regression([log(n) for n in N], mean.(bits_success))
plot(N, mean.(bits_success), grid=false, yerror=std.(bits_success), label="Results", dpi=1000)
plot!(N, [regression[1] + regression[2]*log(n) for n in N], label="$(round(regression[2], digits=1))log(n)")
for i in 1:length(bits_success)
    scatter!([N[i] for _ in 1:length(bits_success[i])], bits_success[i], label="")
end

title!("WL vs 1D-MPGNN, Erdős Rényi")
xlabel!("N")
ylabel!("Message Size (Bits)")
savefig("results\\2.bits_erdos_renyi.png")

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\Single-channel-GNN\\results\\2.bits_erdos_renyi.png"

In [ ]:
regression = Regression([log(n) for n in N], mean.(bits_success))
plot(N, mean.(bits_success), grid=false, yerror=std.(bits_success), xaxis=:log, label="Results", dpi=1000)
plot!(N, [regression[1] + regression[2]*log(n) for n in N], label="$(round(regression[2], digits=1))log(n)")
for i in 1:length(bits_success)
    scatter!([N[i] for _ in 1:length(bits_success[i])], bits_success[i], label="")
end

title!("WL vs 1D-MPGNN, Erdős Rényi")
xlabel!("N")
ylabel!("Message Size (Bits)")
savefig("results\\2.bits_erdos_renyi.png")

## 3. Big Erdős Rényi Experiment

In [119]:
graph = erdos_renyi(5000, 0.1)
graph = graph[findall(>(0), degree(graph))]
x₀, edge_index = construct_graph(graph)

iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
wl_classes = maximum(coloring_wl)

4984.0

In [120]:
setprecision(BigFloat, 60)
bits = [i for i ∈ 10: 100: 800]
γs = sort([rand(BigFloat) for _ in 1:10])
classes = [[] for bit in bits]

for (i, bit) ∈ enumerate(bits)
    setprecision(BigFloat, bit)
    for γ ∈ γs
        coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
        push!(classes[i], maximum(coloring_gnn))
    end
end

plot(bits, [wl_classes for _ in bits], label="WL", dpi=1000)
plot!(bits, mean.(classes), grid=false, yerror=std.(classes), label="GNN")
title!("WL vs 1D-MPGNN, Big Erdős Rényi")
xlabel!("Message Size (Bits)")
ylabel!("Number of classes")
savefig("results\\3.big_erdos_renyi.png")

"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\Single-channel-GNN\\results\\3.big_erdos_renyi.png"

## 4. Cora Experiment

In [98]:
dataset_directory = "data/"
dataset = pyg.datasets.Planetoid(dataset_directory, "Cora")
graph = dataset[0]
x₀, edge_index = ones(2708), graph.edge_index.numpy() .+ 1

iters, coloring_wl = WeisfeilerLehman(x₀, edge_index)
wl_classes = maximum(coloring_wl)

([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1 1 … 2708 2708; 634 1863 … 1474 2707])

In [116]:
setprecision(BigFloat, 60)
bits = [i for i ∈ 10: 1 :60]
γs = sort([rand(BigFloat) for _ in 1:10])
classes = [[] for bit in bits]

for (i, bit) ∈ enumerate(bits)
    setprecision(BigFloat, bit)
    for γ ∈ γs
        coloring_gnn = MPGNN(x₀, edge_index, γ, iters)
        push!(classes[i], maximum(coloring_gnn))
    end
end

plot(bits, [wl_classes for _ in bits], label="WL", dpi=1000)
plot!(bits, mean.(classes), grid=false, yerror=std.(classes), label="GNN")
title!("WL vs 1D-MPGNN, Cora")
xlabel!("Message Size (Bits)")
ylabel!("Number of classes")
savefig("results\\4.cora.png")


"c:\\Users\\jacmo\\OneDrive - uc.cl\\Documentos\\cosas de la u\\Magister\\Single-channel-GNN\\results\\4.cora.png"

In [118]:
display(mean.(classes)[end-10:end])

11-element Vector{BigFloat}:
 2359.0
 2359.1999999999999993
 2359.6999999999999993
 2360.6000000000000014
 2361.0
 2362.6000000000000014
 2363.1999999999999993
 2364.1999999999999993
 2363.6999999999999993
 2364.5
 2365.1000000000000014